# Analysis of CTA Ridership data

In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('cta_ridership.db')

### Check that data has been loaded into tables as expected

In [5]:
# Annual ridership data
annual_ridership_query = """
SELECT * FROM annual_boarding_totals;
"""

annual_df = pd.read_sql(annual_ridership_query, conn)
annual_df

,year,bus,paratransit,rail,total
0,1988,"430,089,500","435,400","174,436,000","604,960,900"
1,1989,"420,572,700","924,800","168,658,800","590,156,300"
2,1990,"421,183,734","930,802","165,732,575","587,847,111"
3,1991,"392,088,602","949,460","147,608,116","540,646,178"
4,1992,"370,335,119","1,011,669","137,372,830","508,719,618"
5,1993,"326,655,953","1,167,904","135,369,734","463,193,591"
6,1994,"331,520,700","1,209,900","143,579,100","476,309,700"
7,1995,"306,075,585","1,270,274","135,461,619","442,807,478"
8,1996,"302,115,116","1,244,209","142,040,486","445,399,811"
9,1997,"287,628,293","1,235,085","151,010,374","439,873,752"


In [3]:
### Daily Bus Ridership Data
bus_query = """
SELECT * FROM daily_ridership_bus_routes;
"""

bus_df = pd.read_sql(bus_query, conn)
bus_df

,route,date,daytype,rides
0,1,01/02/2001,W,"5,813"
1,1,01/03/2001,W,"6,809"
2,1,01/04/2001,W,"6,907"
3,1,01/05/2001,W,"6,154"
4,1,01/08/2001,W,"6,126"
...,...,...,...,...
1081968,X99,03/21/2005,W,55
1081969,X99,03/22/2005,W,58
1081970,X99,03/23/2005,W,49
1081971,X99,03/24/2005,W,53


In [4]:
# Daily L (train) ridership data
train_ridership_query = """
SELECT * FROM daily_ridership_l_stations;
"""

train_df = pd.read_sql(train_ridership_query, conn)
train_df

,station_id,stationname,date,daytype,rides
0,40350,UIC-Halsted,01/01/2001,U,273
1,41130,Halsted-Orange,01/01/2001,U,306
2,40760,Granville,01/01/2001,U,"1,059"
3,40070,Jackson/Dearborn,01/01/2001,U,649
4,40090,Damen-Brown,01/01/2001,U,411
...,...,...,...,...,...
1284672,41670,Conservatory,08/31/2025,U,558
1284673,41680,Oakton-Skokie,08/31/2025,U,250
1284674,41690,Cermak-McCormick Place,08/31/2025,U,"1,459"
1284675,41700,Washington/Wabash,08/31/2025,U,"6,586"


## Begin Analysis

### Q1: How has ridership for the L and Buses changed 5 years before and 5 years after Covid-19?

In [33]:
# analysis starts on 11/01/2014 and ends on 11/01/2025 due to 2025's data availability
covid_ridership_impacts_query = """
SELECT sum(train.rides) as train_ride_count, sum(bus.rides) as bus_ride_count
    FROM daily_ridership_l_stations train
    INNER JOIN daily_ridership_bus_routes bus
        ON train.date = bus.date
    WHERE train.date >= '11/01/2014' AND train.date <= '11/01/2024'
"""

covid_ridership_df = pd.read_sql(covid_ridership_impacts_query, conn)
covid_ridership_df

,train_ride_count,bus_ride_count
0,27359441.0,20678280.0


In [23]:
# analysis starts on 11/01/2014 and ends on 11/01/2025 due to 2025's data availability
covid_ridership_impacts_query = """
with total_ridership as
   (SELECT train.date as date, train.rides as train_ride_count, bus.rides as bus_ride_count
    FROM daily_ridership_l_stations train
    INNER JOIN daily_ridership_bus_routes bus
        ON train.date = bus.date
    WHERE train.date >= '11/01/2014' AND train.date <= '11/01/2025')
SELECT 'pre-pandemic' as timeframe, sum(train_ride_count), sum(bus_ride_count) FROM total_ridership WHERE date < '01/01/2020'
UNION
SELECT 'post-pandemic' as timeframe, sum(train_ride_count), sum(bus_ride_count) FROM total_ridership WHERE date >= '01/01/2020'
"""

covid_ridership_df = pd.read_sql(covid_ridership_impacts_query, conn)
covid_ridership_df

,timeframe,sum(train_ride_count),sum(bus_ride_count)
0,post-pandemic,27359441.0,20678280.0
1,pre-pandemic,NaN,NaN
